### Data exploration


In [7]:
from data_processing import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
X, y = get_data()
X.head()
#Unnamed: 0 is the Id of the row


,Unnamed: 0,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,Age,Education,Income,BMI_Category,Healthy_Diet,Mental_Health_Risk,Heart_Disease_Risk,Age_Group,Education_Level,Income_Group
0,31141,0,1,1,20.693674,1,0,0,1,1,...,11.725312,6.0,8.0,Normal weight,1,0,2,Young,Some College,High Income
1,98230,0,0,1,28.375985,0,0,0,1,0,...,7.977719,5.0,8.0,Obese,0,0,0,Young,Some College,High Income
2,89662,1,1,1,24.629353,0,0,0,1,1,...,8.161838,5.0,6.0,Normal weight,1,0,2,Young,Some College,Middle Income
3,208255,0,1,1,27.246610,0,0,0,1,1,...,5.038298,6.0,7.0,Overweight,1,0,1,Young,Some College,High Income
4,233415,0,1,1,23.671678,0,0,0,1,1,...,12.037360,4.0,6.0,Normal weight,1,0,1,Young,High School Graduate,Middle Income


In [17]:
print('Number of elements :', len(X))
print('Number of features :', len(X.columns))

Number of elements : 202944
Number of features : 29


In [10]:
y.head()

,Unnamed: 0,Diabetes_binary
0,31141,0
1,98230,0
2,89662,1
3,208255,0
4,233415,0


In [11]:
print('len(y):', len(y))

len(y): 202944


### Process the data

In [ ]:
# Look for multicollinearity between the features
# we already know the age so age_group is not needed
X = X.drop(columns=['Unnamed: 0', 'Age_group'])
correlation_matrix = X.corr() #TODO


In [13]:
X = feature_encoding(X)

   Unnamed: 0  HighBP  HighChol  CholCheck        BMI  Smoker  Stroke  \
0       31141       0         1          1  20.693674       1       0   
1       98230       0         0          1  28.375985       0       0   
2       89662       1         1          1  24.629353       0       0   
3      208255       0         1          1  27.246610       0       0   
4      233415       0         1          1  23.671678       0       0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...        Age  Education  \
0                     0             1       1  ...  11.725312        6.0   
1                     0             1       0  ...   7.977719        5.0   
2                     0             1       1  ...   8.161838        5.0   
3                     0             1       1  ...   5.038298        6.0   
4                     0             1       1  ...  12.037360        4.0   

   Income  BMI_Category  Healthy_Diet  Mental_Health_Risk  Heart_Disease_Risk  \
0     8.0             0